In [ ]:
#Importing Data Manipulation Libraries
import numpy as np
import pandas as pd
#Importing Data Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
# Importing Warnings
import warnings 
warnings.filterwarnings('ignore')
#Importing Scikit-Learn Library
from sklearn.preprocessing import MinMaxScaler,RobustScaler,StandardScaler
from sklearn.model_selection import train_test_split;
#Importing Metrics for Regression Model
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.metrics import confusion_matrix
#Importing Logging
import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    filemode='w',
                    filename='model.log',force=True)


In [4]:
url = 'https://raw.githubusercontent.com/meet-vasita/Telemarketing_Classification_Model/refs/heads/main/Dataset/BankData.csv'

df = pd.read_csv(url,sep=';')
df.sample(frac=1)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
4035,47,technician,married,secondary,no,2246,yes,no,cellular,10,jul,330,1,-1,0,unknown,no
1153,41,management,divorced,tertiary,no,369,yes,no,telephone,22,aug,9,3,-1,0,unknown,no
4246,32,services,single,secondary,no,0,yes,no,cellular,31,jul,99,13,-1,0,unknown,no
4107,50,blue-collar,married,primary,no,2142,yes,no,cellular,29,jan,309,2,224,1,other,no
3373,28,admin.,single,tertiary,no,14,no,no,cellular,1,jun,201,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,40,entrepreneur,married,secondary,no,1416,yes,no,unknown,14,may,210,2,-1,0,unknown,no
3731,52,retired,married,secondary,no,424,no,no,cellular,19,nov,143,1,154,3,failure,no
2444,35,management,married,secondary,no,724,yes,no,cellular,14,jul,287,2,415,1,failure,yes
2027,38,blue-collar,single,primary,no,490,yes,yes,unknown,25,aug,17,1,-1,0,unknown,no


In [5]:
logging.info('Importing the dataset for model building')